In [23]:
import os
import csv
from collections import defaultdict
import json

# To finetune

In [45]:
DATASET_VAR = 'augm_wabad'
DATASET_NAME = 'dataset'
RESULTS_PATH = f'/home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/{DATASET_VAR}'
VALID_TABLE = f'{RESULTS_PATH}/valid/BirdNET_SelectionTable.txt'
TRAIN_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/train'
VALID_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/valid'
TEST_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/test'
TEST_TABLE = f'{RESULTS_PATH}/test/BirdNET_SelectionTable.txt'

In [46]:
print(f'python -m birdnet_analyzer.train --i {TRAIN_PATH} --o {RESULTS_PATH}/{DATASET_VAR}.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file {RESULTS_PATH}/{DATASET_VAR}.npz')

python -m birdnet_analyzer.train --i /home/giacomoschiavo/segments/dataset/train --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_wabad/augm_wabad.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_wabad/augm_wabad.npz


# To analize

In [47]:
MIN_CONF = '0.05'

In [48]:
print(f'python -m birdnet_analyzer.analyze --i {VALID_PATH} --o {RESULTS_PATH}/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/valid --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_wabad/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_wabad/augm_wabad.tflite


In [49]:
print(f'python -m birdnet_analyzer.analyze --i {TEST_PATH} --o {RESULTS_PATH}/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/test --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_wabad/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_wabad/augm_wabad.tflite


# Analysis

In [66]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/species_dict_map.json') as f:
    species_dict = json.load(f)
inv_species_dict = {value: key for key, value in species_dict.items()}

In [67]:
def get_conf_scores():
    conf_scores = {}

    #  {'Aeroplane': [(np.float32(0.0), False),
    #               (np.float32(1.3937646e-30), False),
    #               (np.float32(1.0654355e-25), False),
    #               (np.float32(0.0), False),

    with open(VALID_TABLE, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
        for row in reader:
            file_path = row['Begin Path']
            pred_species_name = row['Common Name']
            if row['Common Name'] in inv_species_dict:
                pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
            true_species_name = file_path.split('/')[-2]
            confidence = float(row['Confidence'])
            if pred_species_name not in conf_scores:
                conf_scores[pred_species_name] = []
            is_correct = pred_species_name == true_species_name
            conf_scores[pred_species_name].append((confidence, is_correct))
    return conf_scores
            

In [68]:
conf_scores = get_conf_scores()

In [69]:
from sklearn.metrics import f1_score
import numpy as np 

DEFAULT_THRESH = 0.25

def compute_best_thresholds(conf_scores, num_thresholds=200, min_thresh=0.001, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = DEFAULT_THRESH
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [70]:
best_thresholds = compute_best_thresholds(conf_scores)

📊 Phylloscopus collybita_Common Chiffchaff -> 0.053, F1-score: 0.070
📊 Sylvia atricapilla_Eurasian Blackcap -> 0.250, F1-score: 0.000
📊 Fringilla coelebs_Common Chaffinch -> 0.001, F1-score: 0.489
📊 Troglodytes troglodytes_Eurasian Wren -> 0.250, F1-score: 0.000
📊 Turdus merula_Eurasian Blackbird -> 0.344, F1-score: 0.164
📊 Certhia familiaris_Eurasian Treecreeper -> 0.940, F1-score: 0.819
📊 Lophophanes cristatus_Crested Tit -> 0.082, F1-score: 0.154
📊 Muscicapa striata_Spotted Flycatcher -> 0.250, F1-score: 0.000
📊 Wind -> 0.063, F1-score: 0.222
📊 Loxia curvirostra_Common Crossbill -> 0.154, F1-score: 0.514
📊 Anthus trivialis_Tree Pipit -> 0.850, F1-score: 0.376
📊 Periparus ater_Coal Tit -> 0.721, F1-score: 0.476
📊 Regulus ignicapilla_Common Firecrest -> 0.001, F1-score: 0.168
📊 Regulus regulus_Goldcrest -> 0.249, F1-score: 0.722
📊 Erithacus rubecula_European Robin -> 0.001, F1-score: 0.242
📊 Coccothraustes coccothraustes_Hawfinch -> 0.263, F1-score: 0.412
📊 None -> 0.001, F1-score: 0.

In [71]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_test.json') as f:
    true_segments_test = json.load(f)

In [72]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_train.json') as f:
    true_segments_full = json.load(f)

In [73]:
true_segments_full.update(true_segments_test)

In [74]:
pred_segments = {}
#  {'Aeroplane': [(np.float32(0.0), False),
#               (np.float32(1.3937646e-30), False),
#               (np.float32(1.0654355e-25), False),
#               (np.float32(0.0), False),

with open(TEST_TABLE, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
    for row in reader:
        file_path = row['Begin Path']
        audio_name = os.path.basename(file_path)    # in test non sono presenti WABAD o augm
        only_audio_name = "_".join(audio_name.split("_")[:2]) + ".WAV"
        segm = "_".join(audio_name.split(".")[0].split("_")[-2:])
        pred_species_name = row['Common Name']
        if row['Common Name'] in inv_species_dict:
            pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
        # if len(file_path.split('/')[-2].split('_')) == 1:       # skip if None, Pecking, etc. 
        #     continue
        confidence = float(row['Confidence'])
        if only_audio_name not in pred_segments:
            pred_segments[only_audio_name] = {}
        if pred_species_name not in best_thresholds:
            best_thresholds[pred_species_name] = DEFAULT_THRESH
        if confidence >= best_thresholds[pred_species_name]:
            if segm not in pred_segments[only_audio_name]:
                pred_segments[only_audio_name][segm] = set()
            if "None" in pred_segments[only_audio_name][segm]:
                continue
            if pred_species_name == "None":
                pred_segments[only_audio_name][segm] = set(["None"])
            else:
                pred_segments[only_audio_name][segm].add(pred_species_name)
            

In [75]:
from sklearn.preprocessing import MultiLabelBinarizer

test_species_list = os.listdir(TEST_PATH)
# test_species_list = [species for species in test_species_list if len(species.split('_')) > 1]
mlb = MultiLabelBinarizer()
mlb.fit([test_species_list])

len(mlb.classes_)

21

In [76]:
true_segments = defaultdict(dict)

for species in os.listdir(TEST_PATH):
    if species not in test_species_list:
        print(species, 'ignored')
        continue
    for audio in os.listdir(os.path.join(TEST_PATH, species)):
        audio = audio.split('.')[0]
        date, time, segm1, segm2 = audio.split('_')
        audio_name = '_'.join([date, time]) + '.WAV'
        segm = '_'.join([segm1, segm2])
        if segm not in true_segments[audio_name]:
            true_segments[audio_name][segm] = []
        true_segments[audio_name][segm].extend([species])

In [77]:
for audio in true_segments.keys():
    if audio not in pred_segments:
        pred_segments[audio] = {}
    for segm in true_segments[audio].keys():
        if segm not in pred_segments[audio]:
            pred_segments[audio][segm] = []
        

In [78]:
y_pred = []
y_true = []
for audio in pred_segments:
    # sort in increasing order
    sortable_true_segments = { str(key): value for key, value in true_segments[audio].items() }
    sortable_pred_segments = { str(key): value for key, value in pred_segments[audio].items() }
    sorted_true_segments = dict(sorted(sortable_true_segments.items()))
    sorted_pred_segments = dict(sorted(sortable_pred_segments.items()))
    y_true.append(mlb.transform(sorted_true_segments.values()))     # apply transform on every label of every segment
    y_pred.append(mlb.transform(sorted_pred_segments.values()))     # apply transform on every label of every segment

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

In [79]:
from sklearn.metrics import classification_report
import pandas as pd
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Aeroplane,0.000000,0.000000,0.000000,22.0
Anthus trivialis_Tree Pipit,0.796296,0.268750,0.401869,160.0
Certhia familiaris_Eurasian Treecreeper,0.304348,0.076087,0.121739,92.0
Coccothraustes coccothraustes_Hawfinch,0.785714,0.126437,0.217822,87.0
Cuculus canorus_Common Cuckoo,0.000000,0.000000,0.000000,3.0
Erithacus rubecula_European Robin,0.456977,0.706835,0.555085,556.0
Fringilla coelebs_Common Chaffinch,0.535380,0.751095,0.625152,1370.0
Lophophanes cristatus_Crested Tit,0.015873,0.043478,0.023256,23.0
Loxia curvirostra_Common Crossbill,0.611111,0.305556,0.407407,36.0
Muscicapa striata_Spotted Flycatcher,0.000000,0.000000,0.000000,173.0


In [80]:
with open(f'{RESULTS_PATH}/classification_report.json', 'w') as f:
    json.dump(report, f)

In [81]:
from sklearn.metrics import multilabel_confusion_matrix

mcm = multilabel_confusion_matrix(y_true, y_pred)

# Per visualizzarle in un DataFrame leggibile:
mlb_labels = mlb.classes_  # nome delle classi, già usato da te

# Costruisco un dataframe
conf_matrices = []
for idx, label in enumerate(mlb_labels):
    tn, fp, fn, tp = mcm[idx].ravel()
    conf_matrices.append({
        'Class': label,
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn
    })

conf_df = pd.DataFrame(conf_matrices)
conf_df


,Class,TP,FP,FN,TN
0,Aeroplane,0,0,22,8953
1,Anthus trivialis_Tree Pipit,43,11,117,8804
2,Certhia familiaris_Eurasian Treecreeper,7,16,85,8867
3,Coccothraustes coccothraustes_Hawfinch,11,3,76,8885
4,Cuculus canorus_Common Cuckoo,0,0,3,8972
5,Erithacus rubecula_European Robin,393,467,163,7952
6,Fringilla coelebs_Common Chaffinch,1029,893,341,6712
7,Lophophanes cristatus_Crested Tit,1,62,22,8890
8,Loxia curvirostra_Common Crossbill,11,7,25,8932
9,Muscicapa striata_Spotted Flycatcher,0,0,173,8802
